# Problem Statement & Solution Design

### **Problem Statement:**

ABC Corporation lacks data-driven into their workforce demographics, compensation structure, and  temporal trends. HR decisions are made based on institution rather than emperical evidence, leading to potential inefficiencies in salary distribution, age-based workforce planning, and category-wise resource allocation.

### **Desired Solution:**
A comprehensive HR Analytics Dashboard that provides:
1. Workforce demographics analysis
2. Compensation structure insights
3. Temporal hiring trends
4. Category-wise performance metrics
5. Interactive exploration capabilities

In [1]:
"""
10000-row dirty dataset generator
Matches the flaws in the 6-row example:
  – duplicate names
  – None / NaN / empty strings
  – negative ages
  – invalid dates
  – missing salaries
  – stray whitespace & mixed-case names
  – rare exotic Unicode
  – leading zeros in IDs
  – occasional extra columns (ragged CSV feel)
"""

import pandas as pd
import numpy as np
from datetime import date, timedelta
import random
import warnings
warnings.filterwarnings('ignore')

RNG = np.random.default_rng(42)
SIZE = 10000

# 1) ID: mostly sequential but with gaps and leading zeros
ids = list(range(1, SIZE + 1))
RNG.shuffle(ids)
ids = [f"{i:05d}" for i in ids]          # 00001, 00002 …

# 2) Names: common, rare, empty, None, Unicode, spaces
first = ["Alice", "Bob", "Charlie", "Diana", "Eve", "Frank", "Grace", None, "", "  ", "bob", "ALICE"]
last  = ["Smith", "Johnson", "Lee", "O'Connor", "van der Waal", "Михайлов", "🐼", None, ""]
names = [f"{RNG.choice(first)} {RNG.choice(last)}" for _ in range(SIZE)]
# sprinkle pure None
for i in RNG.choice(SIZE, size=300, replace=False):
    names[i] = None

# 3) Age: positives, negatives, NaN, floats
ages = RNG.normal(loc=35, scale=12, size=SIZE)
ages[RNG.random(SIZE) < 0.08] = np.nan
ages[RNG.random(SIZE) < 0.02] = RNG.integers(-10, 0)

# 4) Salary: missing, negatives, huge, cents
salaries = RNG.lognormal(mean=11, sigma=0.5, size=SIZE)
salaries[RNG.random(SIZE) < 0.07] = np.nan
salaries[RNG.random(SIZE) < 0.01] = -RNG.exponential(1000)
salaries = np.round(salaries, 2)

# 5) Date: valid, impossible, mis-formatted
start = date(2020, 1, 1)
def random_date():
    if RNG.random() < 0.05:
        return RNG.choice(["invalid", "2021-02-30", "13/45/2022", "", "N/A"])
    delta = timedelta(days=int(RNG.integers(0, 1500)))
    d = start + delta
    if RNG.random() < 0.10:
        return d.strftime("%d/%m/%Y")
    return str(d)

dates = [random_date() for _ in range(SIZE)]

# 6) Category: A B C plus garbage
cats = ["A", "B", "C", None, "", "a", "b", "🎲"]
categories = RNG.choice(cats, size=SIZE, p=[0.3, 0.3, 0.3, 0.03, 0.03, 0.02, 0.01, 0.01])

# 7) Build DataFrame
df = pd.DataFrame({
    "id": ids,
    "name": names,
    "age": ages,
    "salary": salaries,
    "date": dates,
    "category": categories
})

# 8) Extra dirt: duplicate rows, swapped columns, trailing spaces
df = pd.concat([df, df.sample(n=200, random_state=RNG)], ignore_index=True)   # 200 dupes
df["name"] = df["name"].astype(str).str.strip() + RNG.choice(["", " ", "  "], size=len(df))
df.iloc[RNG.choice(len(df), 50), 0] = ""   # blank ids
df.iloc[RNG.choice(len(df), 50), 2] = "thirty-five"  # string in age

# 9) Save
df.to_csv("messy_dataset.csv", index=False)
print("messy_dataset.csv created →", df.shape)
print(df.head(10))

messy_dataset.csv created → (10200, 6)
      id                name        age     salary        date category
0  08133          None Lee    16.741619  102784.00  2022-11-14        B
1           Diana Михайлов    43.415811   48330.35  2020-02-19        a
2  00720        Charlie None  28.195627  126557.90  2023-03-29        A
3  08238           bob Smith  39.515083   71411.52  2022-11-05        A
4  04556        ALICE Smith   45.646664  158614.65  2020-06-20        B
5  06652           Frank 🐼    38.934451        NaN  2022-06-23        B
6  05651       Diana Johnson  44.589521   43432.43  13/45/2022        B
7  09035           Alice 🐼    10.200329   53309.34  2024-01-10        A
8  01261  Alice van der Waal   29.08572   32652.40  2022-04-26        C
9  00645                Eve    48.23692   95124.50  2022-09-05        C


In [2]:
df.head(10)

,id,name,age,salary,date,category
0,08133,None Lee,16.741619,102784.00,2022-11-14,B
1,,Diana Михайлов,43.415811,48330.35,2020-02-19,a
2,00720,Charlie None,28.195627,126557.90,2023-03-29,A
3,08238,bob Smith,39.515083,71411.52,2022-11-05,A
4,04556,ALICE Smith,45.646664,158614.65,2020-06-20,B
5,06652,Frank 🐼,38.934451,NaN,2022-06-23,B
6,05651,Diana Johnson,44.589521,43432.43,13/45/2022,B
7,09035,Alice 🐼,10.200329,53309.34,2024-01-10,A
8,01261,Alice van der Waal,29.08572,32652.40,2022-04-26,C
9,00645,Eve,48.23692,95124.50,2022-09-05,C


In [3]:
df.tail(10)

,id,name,age,salary,date,category
10190,05006,Alice Михайлов,60.029926,45347.51,2023-03-30,B
10191,05316,Diana Smith,36.856868,69587.20,2020-12-30,A
10192,08049,None Johnson,NaN,54285.37,2022-11-07,C
10193,09524,Diana Михайлов,46.04597,60838.92,2020-10-19,A
10194,06246,ALICE None,46.419879,84660.20,,C
10195,02811,Smith,NaN,88323.82,2023-06-13,C
10196,06218,bob van der Waal,26.694159,63904.94,2024-02-02,B
10197,06504,Charlie,32.659107,15803.28,2021-11-10,C
10198,04692,bob Smith,60.143554,50487.89,2023-06-22,A
10199,08958,Charlie,NaN,55136.62,2021-05-06,None


In [4]:
df.dtypes

id           object
name         object
age          object
salary      float64
date         object
category     object
dtype: object

In [5]:
df.size

61200

In [6]:
df.shape

(10200, 6)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10200 entries, 0 to 10199
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        10200 non-null  object 
 1   name      10200 non-null  object 
 2   age       9393 non-null   object 
 3   salary    9524 non-null   float64
 4   date      10200 non-null  object 
 5   category  9923 non-null   object 
dtypes: float64(1), object(5)
memory usage: 478.3+ KB


In [8]:
df.describe()

,salary
count,9524.000000
mean,66424.999071
std,36477.816459
min,-30.040000
25%,41873.285000
50%,58793.045000
75%,82240.572500
max,392309.920000


In [9]:
df.describe(include='all')

,id,name,age,salary,date,category
count,10200,10200,9393.0,9524.000000,10200,9923
unique,9954,294,8971.0,NaN,2164,7
top,,None,-6.0,NaN,13/45/2022,B
freq,50,197,201.0,NaN,119,3111
mean,NaN,NaN,NaN,66424.999071,NaN,NaN
std,NaN,NaN,NaN,36477.816459,NaN,NaN
min,NaN,NaN,NaN,-30.040000,NaN,NaN
25%,NaN,NaN,NaN,41873.285000,NaN,NaN
50%,NaN,NaN,NaN,58793.045000,NaN,NaN
75%,NaN,NaN,NaN,82240.572500,NaN,NaN


In [10]:
df.isnull().sum()

id            0
name          0
age         807
salary      676
date          0
category    277
dtype: int64

In [11]:
df.isna().sum()

id            0
name          0
age         807
salary      676
date          0
category    277
dtype: int64

In [12]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
10195    False
10196    False
10197     True
10198    False
10199    False
Length: 10200, dtype: bool

In [13]:
df.duplicated().sum()

np.int64(56)

In [14]:
duplicate_rows_df = df[df.duplicated()]
duplicate_rows_df

,id,name,age,salary,date,category
10006,04280,Charlie van der Waal,27.506392,71272.04,2021-06-11,B
10007,05113,Charlie Smith,NaN,72510.83,2021-01-10,C
10009,07007,Grace None,NaN,NaN,2021-11-01,A
10011,01735,Grace van der Waal,NaN,181071.86,2020-12-11,None
10013,04871,Alice Михайлов,37.889299,40807.27,2020-12-05,C
10015,02144,Frank O'Connor,17.876179,35368.76,2020-01-16,C
10016,08127,None O'Connor,51.103991,70605.62,2021-12-09,B
10019,06045,None 🐼,16.329463,109945.56,2023-01-04,B
10021,01918,Bob Johnson,38.242005,49217.47,2022-03-08,C
10023,08210,bob Smith,32.521784,27080.42,24/02/2022,C


# ETL PIPELINE

In [ ]:
import pandas as pd # core data-frame library
import sqlite3 # SQLite connector (for DB source)
import numpy as np # Numerical helpers (stats)
from datetime import datetime # default datetime filler
import pandas.api.types as pd_types # type-checking utilities
import unicodedata # unicode normalisation (NFKC)
import re # Regex for pattern removal
import warnings # Silence harmless warnings
from pathlib import Path # OS-independent path handling
from tqdm import tqdm # progress bars (unused now)

warnings.filterwarnings('ignore') # Suppress pandas SettingWithCopy, etc.
pd.set_option('display.max_columns', None) # show all columns in .info()

def etl_pipeline(
    source_path: str,
    is_csv: bool = True,
    table_name: str = None,
    datetime_columns: list = None,
    numeric_columns_for_outliers: list = None,
    numeric_columns_for_standardization: list = None,
    numeric_prep_columns: list = None,
    categorical_columns: list = None,
    output_path: str = 'cleaned_data.csv',
    fill_datetime_na_with: datetime = None,
    type_conversions: dict = None,
    normalize_text: bool = True,
    remove_patterns: list = True,
    auto_create_table: bool = False,
    source_csv_for_create: str = None
):